In [1]:
%%capture
import scrapy
import re
import dateparser
import json
import os
import pandas as pd
import pickle
import pandas as pd
import spacy
import unicodedata
import nltk
import re
import textacy
import contractions
import itertools
import json
import retinasdk
import pyLDAvis.gensim
import numpy as np
from datetime import datetime
from scrapy.crawler import CrawlerProcess
from gensim import corpora, models
from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer
from keras.models import model_from_json
from scipy import spatial
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from twisted.internet import reactor, defer
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging

In [2]:
#helper method to remove all html tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def cleanhelpfulvotes(votes_string):
    vote = votes_string.partition(" ")[0]
    if vote == 'One':
        vote = '1'
    elif len(vote) > 3:
        vote = vote.replace(",", "")
    return int(vote)

In [3]:
#global variables set by the first crawler
image_src = ''
reviews_Link = ''
filename = ''

In [4]:
class ReviewNavSpider(scrapy.Spider):
    name = 'reviewspider'
    start_urls = [
    ]

    def parse(self, response):
        global image_src
        global reviews_Link
        review_dict = []
        image_src = response.xpath('//img[contains(@id, "landingImage")]/@data-old-hires')
        image_src = image_src.extract()[0]
        reviews_Link = response.xpath('//a[contains(@data-hook, "see-all-reviews-link-foot")]/@href')
        reviews_Link = "https://www.amazon.com"+reviews_Link.extract()[0]

In [5]:
class ReviewSpider(scrapy.Spider):
    name = 'reviewspider'
    start_urls = []
    
    def parse(self, response):
        review_dict = []
        reviews = response.xpath('//div[@data-hook="review"]')
        article = response.xpath(
            '//div[contains(@class, "a-row product-title")]//a[contains(@data-hook, "product-link")]/text()'
        ).extract()
        price = response.xpath(
            '//span[contains(@class, "a-color-price arp-price")]/text()'
        ).extract()
        for review in reviews:
            review_title = review.xpath(
                './/a[contains(@data-hook, "review-title")]/text()').extract()
            review_rating = review.xpath(
                './/i[contains(@data-hook, "review-star-rating")]/span[contains(@class, "a-icon-alt")]/text()'
            ).extract()
            review_date = review.xpath(
                './/span[contains(@data-hook, "review-date")]/text()').extract(
                )
            review_body = review.xpath(
                './/span[contains(@data-hook, "review-body")]').extract()
            review_helpful_votes = review.xpath(
                './/span[contains(@data-hook, "helpful-vote-statement")]/text()'
            ).extract()
            review_source = review.xpath(
                './/a[contains(@data-hook, "review-title")]/@href').extract()
            review = {
                "article":
                article[0],
                "price":
                float(price[0].strip().lstrip('$')),
                "review_title":
                review_title[0],
                "review_rating":
                int(review_rating[0][0]),
                "review_date":
                dateparser.parse(review_date[0]).strftime('%Y-%m-%d'),
                "review_body":
                cleanhtml(review_body[0]),
                "review_helpful_votes":
                cleanhelpfulvotes(review_helpful_votes[0])
                if review_helpful_votes else 0,
                "review_source":
                "https://www.amazon.com" + review_source[0]
            }
            review_dict.append(review)
        global filename
        filename = "data/{}.json".format(re.sub('[^a-zA-z0-9]', '', article[0]).lower())
        if not os.path.isfile(filename):
            with open(filename, mode='w') as f:
                f.write(json.dumps([]))
        with open(filename) as feedsjson:
            feeds = json.load(feedsjson)
            feeds.append(review_dict)
        with open(filename, mode='w') as f:
            f.write(json.dumps(feeds))
        yield response.follow(
            "https://www.amazon.com" +
            response.xpath('//li[@class="a-last"]/a/@href').extract()[0],
            callback=self.parse)

In [6]:
#%%capture
# crawl sequentially
configure_logging()
runner = CrawlerRunner({
    'DOWNLOAD_DELAY':
    '2',
    'CONCURRENT_REQUESTS':
    '1',
    'CONCURRNT_REQUESTS_PER_IP':
    '1'
})

@defer.inlineCallbacks
def crawl(start_url):
    spider1.start_urls.append(start_url)
    yield runner.crawl(ReviewNavSpider)
    spider2.start_urls.append(reviews_Link+'&pageNumber=1')
    yield runner.crawl(spider2)
    reactor.stop()
    
spider1 = ReviewNavSpider()
spider2 = ReviewSpider()

In [ ]:
#start_url = 'https://www.amazon.com/dp/B00BGO0Q9O/ref=twister_B00BR00AWU?th=1'
#crawl(start_url)
#reactor.run()